# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv("cal_housing_clean.csv")

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
X = housing.drop("medianHouseValue",axis=1)

In [6]:
y = housing["medianHouseValue"]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler_model = MinMaxScaler()

In [11]:
scaler_model.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler_model.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler_model.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column("housingMedianAge")
rooms = tf.feature_column.numeric_column("totalRooms")
bedrooms = tf.feature_column.numeric_column("totalBedrooms")
pop = tf.feature_column.numeric_column("population")
households = tf.feature_column.numeric_column("households")
income = tf.feature_column.numeric_column("medianIncome")

In [17]:
feat_cols = [age, rooms, bedrooms, pop, households, income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_fun = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [19]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmpzx_357em', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_fun,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmpzx_357em/model.ckpt.
INFO:tensorflow:loss = 3.02288e+11, step = 1
INFO:tensorflow:global_step/sec: 436.152
INFO:tensorflow:loss = 8.28562e+11, step = 101 (0.232 sec)
INFO:tensorflow:global_step/sec: 412.993
INFO:tensorflow:loss = 3.27758e+11, step = 201 (0.243 sec)
INFO:tensorflow:global_step/sec: 342.857
INFO:tensorflow:loss = 6.02963e+11, step = 301 (0.292 sec)
INFO:tensorflow:global_step/sec: 354.206
INFO:tensorflow:loss = 4.07212e+11, step = 401 (0.285 sec)
INFO:tensorflow:global_step/sec: 373.602
INFO:tensorflow:loss = 8.18257e+11, step = 501 (0.265 sec)
INFO:tensorflow:global_step/sec: 369.278
INFO:tensorflow:loss = 7.52138e+11, step = 601 (0.271 sec)
INFO:tensorflow:global_step/sec: 410.468
INFO:tensorflow:loss = 3.97188e+11, step = 701 (0.240 sec)
INFO:tensorflow:global_step/sec: 376.094
INFO:tensorflow:loss = 2.7939e+11, step = 801 (0.26

INFO:tensorflow:loss = 1.00801e+11, step = 8001 (0.275 sec)
INFO:tensorflow:global_step/sec: 361.653
INFO:tensorflow:loss = 5.63687e+10, step = 8101 (0.276 sec)
INFO:tensorflow:global_step/sec: 362.354
INFO:tensorflow:loss = 1.31011e+11, step = 8201 (0.276 sec)
INFO:tensorflow:global_step/sec: 434.005
INFO:tensorflow:loss = 5.36825e+10, step = 8301 (0.230 sec)
INFO:tensorflow:global_step/sec: 351.195
INFO:tensorflow:loss = 1.10555e+11, step = 8401 (0.289 sec)
INFO:tensorflow:global_step/sec: 355.052
INFO:tensorflow:loss = 1.36698e+11, step = 8501 (0.277 sec)
INFO:tensorflow:global_step/sec: 363.247
INFO:tensorflow:loss = 4.08767e+10, step = 8601 (0.275 sec)
INFO:tensorflow:global_step/sec: 409.474
INFO:tensorflow:loss = 1.66553e+11, step = 8701 (0.244 sec)
INFO:tensorflow:global_step/sec: 367.177
INFO:tensorflow:loss = 1.861e+11, step = 8801 (0.277 sec)
INFO:tensorflow:global_step/sec: 355.727
INFO:tensorflow:loss = 1.13003e+11, step = 8901 (0.276 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 7.75756e+10, step = 16101 (0.275 sec)
INFO:tensorflow:global_step/sec: 358.775
INFO:tensorflow:loss = 4.94867e+10, step = 16201 (0.280 sec)
INFO:tensorflow:global_step/sec: 364.556
INFO:tensorflow:loss = 9.78021e+10, step = 16301 (0.273 sec)
INFO:tensorflow:global_step/sec: 402.697
INFO:tensorflow:loss = 7.24111e+10, step = 16401 (0.248 sec)
INFO:tensorflow:global_step/sec: 400.61
INFO:tensorflow:loss = 3.79844e+10, step = 16501 (0.251 sec)
INFO:tensorflow:global_step/sec: 341.732
INFO:tensorflow:loss = 1.2209e+11, step = 16601 (0.297 sec)
INFO:tensorflow:global_step/sec: 344.172
INFO:tensorflow:loss = 1.44931e+11, step = 16701 (0.285 sec)
INFO:tensorflow:global_step/sec: 362.097
INFO:tensorflow:loss = 2.4615e+11, step = 16801 (0.276 sec)
INFO:tensorflow:global_step/sec: 407.896
INFO:tensorflow:loss = 2.46741e+11, step = 16901 (0.245 sec)
INFO:tensorflow:global_step/sec: 415.503
INFO:tensorflow:loss = 5.41998e+10, step = 17001 (0.241 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 284.912
INFO:tensorflow:loss = 5.54859e+10, step = 24201 (0.352 sec)
INFO:tensorflow:global_step/sec: 294.378
INFO:tensorflow:loss = 3.72361e+10, step = 24301 (0.351 sec)
INFO:tensorflow:global_step/sec: 310.431
INFO:tensorflow:loss = 7.19367e+10, step = 24401 (0.315 sec)
INFO:tensorflow:global_step/sec: 280.979
INFO:tensorflow:loss = 3.93174e+10, step = 24501 (0.360 sec)
INFO:tensorflow:global_step/sec: 316.402
INFO:tensorflow:loss = 9.06301e+10, step = 24601 (0.312 sec)
INFO:tensorflow:global_step/sec: 264.49
INFO:tensorflow:loss = 7.36578e+10, step = 24701 (0.374 sec)
INFO:tensorflow:global_step/sec: 359.602
INFO:tensorflow:loss = 6.48113e+10, step = 24801 (0.285 sec)
INFO:tensorflow:global_step/sec: 268.407
INFO:tensorflow:loss = 7.76541e+10, step = 24901 (0.370 sec)
INFO:tensorflow:Saving checkpoints for 25000 into /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmpzx_357em/model.ckpt.
INFO:tensorflow:Loss for final step: 6.48609e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [21]:
predict_input_fun = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [22]:
predictions = model.predict(input_fn=predict_input_fun)

In [23]:
predictions

<generator object Estimator.predict at 0x11733cca8>

In [24]:
predictions = list(predictions)

INFO:tensorflow:Restoring parameters from /var/folders/7b/2llsq3xs5wv7lqvjqk93v0100000gp/T/tmpzx_357em/model.ckpt-25000


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [25]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(y_test,final_preds)**0.5

97818.494400881347

# Great Job!